# How to use

Just execute the commands in order. You will need a valid geckodriver (for licensing purposes, not provided with this notebook)

In [146]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pprint

from bs4 import BeautifulSoup

# Start the driver

You need to set executable path of the geckodriver in executable_path. The geckodriver that is correct for your processor architecture can be found here: https://github.com/mozilla/geckodriver/releases




In [147]:
driver = webdriver.Firefox(executable_path='./geckodriver')
driver.implicitly_wait(5)
#ClimaTempo unfortunately takes a really long time to fully load, so we put a small timeout and catch the tiemoutException
driver.set_page_load_timeout(3)

# Load the first entry  point and acquire the states and cities data

In [148]:
try:
    #first entry point provided
    driver.get("https://www.climatempo.com.br/climatologia/2/santos-sp") 
except TimeoutException:
    pass

def read_states(page):
    # Wait to load the dropdown list
    WebDriverWait(driver, 20).until(
        EC.invisibility_of_element_located((By.XPATH, "//select[@id='sel-state-geo']/option[@value='']")))
    WebDriverWait(driver, 5).until(
        EC.invisibility_of_element_located((By.XPATH, "//select[@id='sel-state-geo']/option[@value='0']")))
    
    states_element = page.find_element_by_id("sel-state-geo")   
    states = [x for x in states_element.find_elements_by_tag_name("option")]
    return states



def read_cities(states):
    cities = []
    for state in states:
        if len(cities) >= 100:
            print("Getting at least the only first 100 to faster processing...")
            return cities
        print("State loaded: {}({})".format(state.text, state.get_attribute("value")))
        state.click()
        
        # Wait to load the dropdown list
        WebDriverWait(driver, 5).until(
            EC.invisibility_of_element_located((By.XPATH, "//select[@id='sel-city-geo']/option[@value='0']")))
        
        cities_element_selector = driver.find_element_by_id("sel-city-geo")
        cities_elements = [x for x in cities_element_selector.find_elements_by_tag_name("option")]
        
        for city in cities_elements:
            cities.append((
                city.get_attribute("value")
                ,{ 
                "State": state.text,
                "City": city.text,
            }))
        
        
    return cities

elem = driver.find_element_by_xpath("/html/body/div[5]/div/div[2]/div[2]/div/div[2]/div[1]/div[1]/p[1]/span[2]")
elem.click()

states = read_states(driver)
cities = read_cities(states)


State loaded: Acre(AC)
State loaded: Alagoas(AL)
Getting at least the only first 100 to faster processing...


# Scrap into pages to acquire the data

In [149]:
def read_city_climatology(code, data):
    print("Scrapping data for {City}({State})".format(**data))
    try:
        #first entry point provided
        driver.get("https://www.climatempo.com.br/climatologia/{}/dummy".format(code)) 
    except TimeoutException:
        pass
        
    table_xpath = "/html/body/div[5]/div/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[3]/table/tbody"
    
    table_soup = BeautifulSoup(driver.find_element_by_xpath(table_xpath).get_attribute('outerHTML'))
    
    
    lines = table_soup.findChildren('tr')
    
    data['Climatology'] = {}
    
    for columns in lines:
        month = ''
        min_temp = 0
        max_temp = 0
        precipitation = 0
        for counter, column in enumerate(columns.findChildren('td')):

            
            if counter == 0:
                month = column.string
            elif counter == 1:
                min_temp = int(column.string.replace(u'\N{DEGREE SIGN}', ''))
            elif counter == 2:
                max_temp = int(column.string.replace(u'\N{DEGREE SIGN}', ''))
            elif counter == 3:
                precipitation = int(column.string)
                
        data['Climatology'][month] = {
            'Min': min_temp,
            'Max': max_temp,
            'Precipitation': precipitation
        }
    
       
        
for code, data in cities:
    read_city_climatology(code, data)

Scrapping data for Rio Branco(Acre)
Scrapping data for Acrelândia(Acre)
Scrapping data for Assis Brasil(Acre)
Scrapping data for Brasiléia(Acre)
Scrapping data for Bujari(Acre)
Scrapping data for Capixaba(Acre)
Scrapping data for Cruzeiro do Sul(Acre)
Scrapping data for Epitaciolândia(Acre)
Scrapping data for Feijó(Acre)
Scrapping data for Jordão(Acre)
Scrapping data for Mâncio Lima(Acre)
Scrapping data for Manoel Urbano(Acre)
Scrapping data for Marechal Thaumaturgo(Acre)
Scrapping data for Plácido de Castro(Acre)
Scrapping data for Porto Acre(Acre)
Scrapping data for Porto Walter(Acre)
Scrapping data for Rodrigues Alves(Acre)
Scrapping data for Santa Rosa do Purus(Acre)
Scrapping data for Senador Guiomard(Acre)
Scrapping data for Sena Madureira(Acre)
Scrapping data for Tarauacá(Acre)
Scrapping data for Xapuri(Acre)
Scrapping data for Maceió(Alagoas)
Scrapping data for Água Branca(Alagoas)
Scrapping data for Anadia(Alagoas)
Scrapping data for Arapiraca(Alagoas)
Scrapping data for Atala

# Print scrapped data

In [151]:
for _, data in cities:
    pprint.pprint(data)


{'City': 'Rio Branco',
 'Climatology': {'Abril': {'Max': 31, 'Min': 21, 'Precipitation': 190},
                 'Agosto': {'Max': 33, 'Min': 17, 'Precipitation': 38},
                 'Dezembro': {'Max': 31, 'Min': 21, 'Precipitation': 265},
                 'Fevereiro': {'Max': 31, 'Min': 21, 'Precipitation': 285},
                 'Janeiro': {'Max': 31, 'Min': 21, 'Precipitation': 289},
                 'Julho': {'Max': 31, 'Min': 17, 'Precipitation': 44},
                 'Junho': {'Max': 30, 'Min': 18, 'Precipitation': 32},
                 'Maio': {'Max': 31, 'Min': 19, 'Precipitation': 93},
                 'Março': {'Max': 31, 'Min': 21, 'Precipitation': 230},
                 'Novembro': {'Max': 32, 'Min': 21, 'Precipitation': 221},
                 'Outubro': {'Max': 33, 'Min': 21, 'Precipitation': 171},
                 'Setembro': {'Max': 33, 'Min': 19, 'Precipitation': 90}},
 'State': 'Acre'}
{'City': 'Acrelândia',
 'Climatology': {'Abril': {'Max': 29, 'Min': 22, 'Precipita

                 'Junho': {'Max': 26, 'Min': 19, 'Precipitation': 79},
                 'Maio': {'Max': 29, 'Min': 20, 'Precipitation': 69},
                 'Março': {'Max': 32, 'Min': 22, 'Precipitation': 84},
                 'Novembro': {'Max': 33, 'Min': 21, 'Precipitation': 19},
                 'Outubro': {'Max': 32, 'Min': 20, 'Precipitation': 14},
                 'Setembro': {'Max': 29, 'Min': 19, 'Precipitation': 19}},
 'State': 'Alagoas'}
{'City': 'Capela',
 'Climatology': {'Abril': {'Max': 29, 'Min': 21, 'Precipitation': 128},
                 'Agosto': {'Max': 24, 'Min': 18, 'Precipitation': 127},
                 'Dezembro': {'Max': 30, 'Min': 21, 'Precipitation': 32},
                 'Fevereiro': {'Max': 30, 'Min': 22, 'Precipitation': 61},
                 'Janeiro': {'Max': 30, 'Min': 21, 'Precipitation': 54},
                 'Julho': {'Max': 24, 'Min': 19, 'Precipitation': 186},
                 'Junho': {'Max': 25, 'Min': 20, 'Precipitation': 195},
               

                 'Julho': {'Max': 23, 'Min': 18, 'Precipitation': 186},
                 'Junho': {'Max': 24, 'Min': 19, 'Precipitation': 195},
                 'Maio': {'Max': 26, 'Min': 20, 'Precipitation': 148},
                 'Março': {'Max': 30, 'Min': 21, 'Precipitation': 93},
                 'Novembro': {'Max': 30, 'Min': 20, 'Precipitation': 25},
                 'Outubro': {'Max': 28, 'Min': 19, 'Precipitation': 35},
                 'Setembro': {'Max': 25, 'Min': 18, 'Precipitation': 70}},
 'State': 'Alagoas'}
{'City': 'Mata Grande',
 'Climatology': {'Abril': {'Max': 31, 'Min': 18, 'Precipitation': 99},
                 'Agosto': {'Max': 26, 'Min': 16, 'Precipitation': 102},
                 'Dezembro': {'Max': 33, 'Min': 18, 'Precipitation': 64},
                 'Fevereiro': {'Max': 32, 'Min': 18, 'Precipitation': 80},
                 'Janeiro': {'Max': 32, 'Min': 18, 'Precipitation': 62},
                 'Julho': {'Max': 26, 'Min': 16, 'Precipitation': 207},
         

                 'Novembro': {'Max': 30, 'Min': 21, 'Precipitation': 36},
                 'Outubro': {'Max': 28, 'Min': 20, 'Precipitation': 49},
                 'Setembro': {'Max': 26, 'Min': 19, 'Precipitation': 85}},
 'State': 'Alagoas'}
{'City': 'Roteiro',
 'Climatology': {'Abril': {'Max': 29, 'Min': 25, 'Precipitation': 213},
                 'Agosto': {'Max': 25, 'Min': 23, 'Precipitation': 152},
                 'Dezembro': {'Max': 29, 'Min': 25, 'Precipitation': 44},
                 'Fevereiro': {'Max': 30, 'Min': 25, 'Precipitation': 86},
                 'Janeiro': {'Max': 29, 'Min': 25, 'Precipitation': 65},
                 'Julho': {'Max': 25, 'Min': 23, 'Precipitation': 243},
                 'Junho': {'Max': 26, 'Min': 24, 'Precipitation': 277},
                 'Maio': {'Max': 28, 'Min': 24, 'Precipitation': 234},
                 'Março': {'Max': 30, 'Min': 25, 'Precipitation': 132},
                 'Novembro': {'Max': 29, 'Min': 24, 'Precipitation': 42},
         